In [ ]:
import sys
sys.path.insert(0, '../../')

import graph
import importlib
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
import schedule as sch
import topo
import typing
import utils
import yaml
import time
from tqdm.notebook import tqdm

matplotlib.rc('font', family='Times New Roman', size=18)

In [ ]:
def unit_size_cb(r: int):
    return 10000 * math.pow(10, random.randint(0, 1))

def timer(f):
    t0 = time.time()
    f()
    t1 = time.time()
    return t1 - t0

def gen_graphs(graph_count, source_selector_dict, max_source_num):
    source_selector = graph.MultiDomainSourceSelector(source_selector_dict)
    global source_quota
    source_quota = max_source_num
    def source_num(i):
        global source_quota
        assert graph_count - i <= source_quota
        delta = source_quota - (graph_count - i)
        ret = 1 + random.randint(0, min(delta, 2))
        source_quota -= ret
        return ret
    gen_args_list = [
            {
            "total_rank": random.randint(3, 7),
            "max_node_per_rank": random.randint(1, 3),
            "max_predecessors": random.randint(1, 2),
            "mi_cb": lambda: 1,
            "memory_cb": lambda: int(2e8),
            "unit_size_cb": unit_size_cb,
            "unit_rate_cb": lambda: random.randint(10, 20),
            "source_hosts": source_selector,
            "sink_hosts": ["cloud1"],
            "sources_num": source_num(i),
        }
        for i in range(graph_count)
    ]
    return [
        graph.MultiSourceGraphGenerator("g" + str(idx), **gen_args).gen_dag_graph()
        for idx, gen_args in enumerate(gen_args_list)
    ]

In [ ]:
def big_avg(array):
    return sum(array) / len(array)

from tqdm.notebook import trange
def run(sc, labels, source_selector_dict, max_source_num):
    flow_time = [[] for _ in labels]
    sbon_time = [[] for _ in labels]
    test_round = 100
    for idx, graph_count in enumerate(labels):
        print(graph_count)
        for _ in trange(test_round, desc="running with {} graphs".format(graph_count)):
            graph_list = gen_graphs(graph_count, source_selector_dict, max_source_num)
            
            sc.topo.clear_occupied()
            flow_scheduler = sch.MultiEdgeFlowScheduler(sc)
            flow_time_1_round = timer(lambda: flow_scheduler.schedule_multiple(graph_list))
            flow_time[idx].append(flow_time_1_round)

            sc.topo.clear_occupied()
            sbon_scheduler = sch.MultiEdgeSbonScheduler(sc)
            sbon_time_1_round = timer(lambda: sbon_scheduler.schedule_multiple(graph_list))
            sbon_time[idx].append(sbon_time_1_round)
            
                
    flow_time = [big_avg(i) for i in flow_time]
    sbon_time = [big_avg(i) for i in sbon_time]
    # return flow_event_data, flow_bp_data, all_cloud_event_data, all_cloud_bp_data, sbon_event_data, sbon_bp_data
    return flow_time, sbon_time

In [ ]:
sc = topo.Scenario.from_dict(yaml.load(open("../../samples/1e6h.yaml", "r").read(), Loader=yaml.Loader))
f_time1, sbon_time1 = run(sc, [4, 8, 12, 16, 20], 
        {"edge0": {'e0rasp1': 10, 'e0rasp2': 10, 'e0rasp3': 10},
        "edge1": {'e1rasp1': 10, 'e1rasp2': 10, 'e1rasp3': 10}}, 45)
print(f_time1)
print(sbon_time1)

In [ ]:
sc = topo.Scenario.from_dict(yaml.load(open("../../samples/1e18h.yaml", "r").read(), Loader=yaml.Loader))
edge0 = {"e0rasp"+str(i): 12 for i in range(1, 7)}
edge1 = {"e1rasp"+str(i): 12 for i in range(1, 4)}
edge1.update({"e1vm"+str(i): 24 for i in range(1, 4)})
edge2 = {"e2rasp"+str(i): 12 for i in range(1, 5)}
edge2.update({"e2vm"+str(i): 24 for i in range(1, 3)})
source_dict = {"edge0": edge0, "edge1": edge1, "edge2": edge2}
f_time2, sbon_time2 = run(sc, [40, 60, 80, 100, 120], source_dict, 200)
print(f_time2)
print(sbon_time2)